In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def sigmoid(x):
    return (1/(1+np.exp(-x)));

class Network(object):
    
    def __init__(self,sizes):
        self.num_layers=len(sizes);
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1],sizes[1:])];
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.sizes=sizes;

    def feedforward(self,a):
        """ Return the output of the whole Network for input ""a"" and weights and biases """
        for w,b in zip(self.weights,self.biases):
            a=sigmoid(np.dot(w,a)+b);
        return a;

    def SGD(self,training_data,epoches,mini_batch_size,eta,test_data=None):
        """Perform Stochastic Gradiant Decent where training data is given in tuples as (x,y) eta is the learning rate"""
        n= len(training_data);
        for j in np.arange(0,epoches):
            training_data=np.random.shuffle(training_data);
            mini_batches=[training_data[k:k+mini_batch_size] for k in np.arange(0,n,mini_batch_size)];

            for mini_batch in mini_batches :
                #self.update_min_batch(mini_batch,eta);
                print("Update Batch")
            
            print("Epoch {0} / {1} completed".format(j,epoches));
    
    def update_mini_batch(mini_batch,eta):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases];
        nabla_w = [np.zeros(w.shape) for w in self.weights];
        
        for (x,y) in mini_batch:
            
            delta_nabla_b, delta_nabla_w = self.backprob(x,y);
            nabla_b = [nb + dnb for nb,dnb in zip(nabla_b,delta_nabla_b)];
            nabla_w = [nw + dnw for nw,dnw in zip(nabla_w,delta_nabla_w)];
        
        self.weights = [w - (eta/len(mini_batch))*nw for w,nw in zip(self.weights,nabla_w) ];
        self.biases = [b - (eta/len(mini_batch))*nb for b,nb in zip(self.biases,nabla_b) ];
        
            
        
               




    